## Solutions for Unit 4 Actions Part 1

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionExercise4-6">Solution Exercise 4.6</a>

## Solution Exercise 4.6 <p id="SolutionExercise4-6"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.6**</p>

For this exercise, we will assume that our package is called **exercise_46**, our launch file is called **move_drone.launch**, and our Python file is called **move_drone.py**.

First of all, you have to make sure that the Action Server is up and running. Otherwise, you won't be able to complete this exercise. For that, you can execute the following command:

In [ ]:
rostopic list | grep ardrone_action_server

If the Action Server is running, you will see this:

<img src="../img/ardrone_as_sol.png" width="600" />

If you don't get this output, it means that your Action Server is not running, so you will have to launch it. For that, just execute the following command:

In [ ]:
roslaunch ardrone_as action_server.launch

When you are sure that the Action Server is running, you can then proceed with the Exercise. Below you can check a possible solution to this Exercise.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: move_drone.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_46" type="move_drone.py" name="drone_action_client" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: move_drone.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: move_drone.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
import time
import actionlib
from ardrone_as.msg import ArdroneAction, ArdroneGoal, ArdroneResult, ArdroneFeedback
from geometry_msgs.msg import Twist
from std_msgs.msg import Empty

# We create some constants with the corresponing vaules from the SimpleGoalState class
PENDING = 0
ACTIVE = 1
DONE = 2
WARN = 3
ERROR = 4

nImage = 1

# definition of the feedback callback. This will be called when feedback
# is received from the action server
# it just prints a message indicating a new message has been received
def feedback_callback(feedback):

    global nImage
    print('[Feedback] image n.%d received'%nImage)
    nImage += 1

# initializes the action client node
rospy.init_node('drone_action_client')

action_server_name = '/ardrone_action_server'
client = actionlib.SimpleActionClient(action_server_name, ArdroneAction)
move = rospy.Publisher('/cmd_vel', Twist, queue_size=1) #Create a Publisher to move the drone
move_msg = Twist() #Create the message to move the drone
takeoff = rospy.Publisher('/drone/takeoff', Empty, queue_size=1) #Create a Publisher to takeoff the drone
takeoff_msg = Empty() #Create the message to takeoff the drone
land = rospy.Publisher('/drone/land', Empty, queue_size=1) #Create a Publisher to land the drone
land_msg = Empty() #Create the message to land the drone

# waits until the action server is up and running
rospy.loginfo('Waiting for action Server '+action_server_name)
client.wait_for_server()
rospy.loginfo('Action Server Found...'+action_server_name)

# creates a goal to send to the action server
goal = ArdroneGoal()
goal.nseconds = 10 # indicates, take pictures along 10 seconds

client.send_goal(goal, feedback_cb=feedback_callback)


# You can access the SimpleAction Variable "simple_state", that will be 1 if active, and 2 when finished.
#Its a variable, better use a function like get_state.
#state = client.simple_state
# state_result will give the FINAL STATE. Will be 1 when Active, and 2 if NO ERROR, 3 If Any Warning, and 3 if ERROR
state_result = client.get_state()

rate = rospy.Rate(1)

rospy.loginfo("state_result: "+str(state_result))

#We takeoff the drone during the first 3 seconds
i=0
while not i == 3:
    takeoff.publish(takeoff_msg)
    rospy.loginfo('Taking off...')
    time.sleep(1)
    i += 1

#We move the drone in a circle wile the state of the Action is not DONE yet
while state_result < DONE:
    move_msg.linear.x = 1
    move_msg.angular.z = 1
    move.publish(move_msg)
    rate.sleep()
    state_result = client.get_state()
    rospy.loginfo('Moving around...')
    rospy.loginfo("state_result: "+str(state_result))
    
rospy.loginfo("[Result] State: "+str(state_result))
if state_result == ERROR:
    rospy.logerr("Something went wrong in the Server Side")
if state_result == WARN:
    rospy.logwarn("There is a warning in the Server Side")

# We land the drone when the action is finished
i=0
while not i == 3:
    move_msg.linear.x = 0
    move_msg.angular.z = 0
    move.publish(move_msg)
    land.publish(land_msg)
    rospy.loginfo('Landing...')
    time.sleep(1)
    i += 1

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: move_drone.py** </p>

So, as you can see in the above code, the logic of the Python script is quite simple:

* During the first 3 seconds, the drone will takeoff.
* Then, it will begin to move doing circles until the Action finishes
* When the Action finished, the drone will land

<img src="../img/move_drone_sol.gif" width="600" />